In [17]:
import csv
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
# Importamos utilidades y modelos de sklearn
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
import category_encoders as ce

In [18]:
dtypes = {'id': 'int32', 
'titulo': 'object', 
'descripcion': 'object', 
'tipodepropiedad': 'category', 
'direccion': 'object', 
'ciudad': 'object', 
'provincia': 'category', 
'antiguedad': 'float', 
'habitaciones': 'float', 
'garages': 'float', 
'banos': 'float', 
'metroscubiertos': 'float', 
'metrostotales': 'float', 
'idzona': 'object', 
'lat': 'float64', 
'lng': 'float64', 
'gimnasio': 'float', 
'usosmultiples': 'float', 
'piscina': 'float', 
'escuelascercanas': 'float', 
'centroscomercialescercanos': 'float', 
'precio': 'float', }

data = pd.read_csv("train.csv", dtype = dtypes)
data.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,23533.0,NaN,NaN,2015-08-23 00:00:00,0.0,0.0,0.0,0.0,0.0,2273000.0
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,24514.0,19.310205,-99.227655,2013-06-28 00:00:00,0.0,0.0,0.0,1.0,1.0,3600000.0
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,48551.0,NaN,NaN,2015-10-17 00:00:00,0.0,0.0,0.0,0.0,0.0,1200000.0
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,53666.0,19.301890,-99.688015,2012-03-09 00:00:00,0.0,0.0,0.0,1.0,1.0,650000.0
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,47835.0,NaN,NaN,2016-06-07 00:00:00,0.0,0.0,0.0,0.0,0.0,1150000.0


# Pre-procesamiento de data

In [19]:
data["ciudad"].nunique()

875

In [20]:
data.isnull().sum()

id                                 0
titulo                          5387
descripcion                     1619
tipodepropiedad                   46
direccion                      53072
ciudad                           372
provincia                        155
antiguedad                     43555
habitaciones                   22471
garages                        37765
banos                          26221
metroscubiertos                17400
metrostotales                  51467
idzona                         28621
lat                           123488
lng                           123488
fecha                              0
gimnasio                           0
usosmultiples                      0
piscina                            0
escuelascercanas                   0
centroscomercialescercanos         0
precio                             0
dtype: int64

In [21]:
data.isnull().sum().sum()

535127

In [22]:
data.size

5520000

In [23]:
(data.isnull().sum().sum()/(data.size))*100

9.694329710144928

Los Nulls representan el 10% de los datos.

# CON XGBOOST NO HACE FALTA PREOCUPARSE POR LOS NULLs

# Levanto el csv de test para calcularle los features en paralelo

In [24]:
test = pd.read_csv("test.csv", dtype = dtypes)
test.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,metrostotales,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos
0,4941,"casa en venta en miguel hidalgo, distrito federal",<p>excelente casa estilo moderno.</p>,Casa,Bosque de Cedros,Miguel Hidalgo,Distrito Federal,29.0,3.0,NaN,...,NaN,NaN,19.408668,-99.246767,2013-07-20 00:00:00,0.0,0.0,0.0,0.0,0.0
1,51775,departamentos en venta en montebello,<p>departamento una recamara:\n</p><p>departam...,Apartamento,NaN,Mérida,Yucatán,NaN,1.0,1.0,...,67.0,113851.0,21.032480,-89.592424,2015-10-24 00:00:00,0.0,0.0,0.0,0.0,0.0
2,115253,departamento nuevo delegación coyoacán de 87 m...,"departamento nuevo de 87.06 m2, 1 cajón de est...",Apartamento,"Pueblo de los Reyes, Coyoacán, Mexico D.F.",Coyoacán,Distrito Federal,0.0,2.0,1.0,...,100.0,23620.0,19.332829,-99.152913,2015-05-30 00:00:00,0.0,0.0,0.0,0.0,1.0
3,299321,departamento en venta en acapulco,<p> raíces dv001 precioso departamento tipo k...,Apartamento,NaN,Acapulco de Juárez,Guerrero,2.0,2.0,2.0,...,86.0,129347.0,16.860487,-99.878383,2015-04-02 00:00:00,0.0,0.0,0.0,0.0,0.0
4,173570,bonita casa sola equipada de dos niveles en lo...,"<p>casa sola, bonita de dos rec&aacute;maras u...",Casa,CEDROS,Tultitlán,Edo. de México,10.0,2.0,1.0,...,76.0,57125.0,19.640482,-99.127273,2013-08-15 00:00:00,0.0,0.0,0.0,1.0,1.0


In [25]:
len(test)

60000

# Preparacion del set de datos de entrenamiento (features)

La idea es preparar el set con los datos para exportar y que despues el modelo simplemente levante, separe en set de entrenamiento y test, entrene y devuelva una prediccion.

Se procede a calcular features. Cada feature se agregara al DataFrame final que tendra que levantar despues el modelo. Ojo que dentro de este DataFrame tambien va a estar el precio (que es el label).

In [26]:
train_set = pd.DataFrame()

In [27]:
test_set = pd.DataFrame()

# Agrego las columnas que ya se encuentran

In [28]:
data.columns

Index(['id', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio'],
      dtype='object')

# Set columnas para 'train_set'

In [29]:
train_cols = ['id','antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio','provincia','ciudad']

In [30]:
train_set = data.loc[:, train_cols]

In [31]:
train_set.head()

,id,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,provincia,ciudad
0,254099,NaN,2.0,1.0,2.0,80.0,80.0,23533.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,2273000.0,Distrito Federal,Benito Juárez
1,53461,10.0,3.0,2.0,2.0,268.0,180.0,24514.0,19.310205,-99.227655,0.0,0.0,0.0,1.0,1.0,3600000.0,Distrito Federal,La Magdalena Contreras
2,247984,5.0,3.0,2.0,2.0,144.0,166.0,48551.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1200000.0,Jalisco,Tonalá
3,209067,1.0,2.0,1.0,1.0,63.0,67.0,53666.0,19.301890,-99.688015,0.0,0.0,0.0,1.0,1.0,650000.0,Edo. de México,Zinacantepec
4,185997,10.0,2.0,1.0,1.0,95.0,95.0,47835.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1150000.0,Jalisco,Zapopan


# Set columnas para 'test_set'

In [32]:
test_cols = ['id', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos','provincia','ciudad']

In [33]:
# Se mete en test_set la columna 'id'; recordar de luego sacarla!
test_set = test.loc[:, test_cols]

In [34]:
test_set.head()

,id,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,provincia,ciudad
0,4941,29.0,3.0,NaN,4.0,300.0,NaN,NaN,19.408668,-99.246767,0.0,0.0,0.0,0.0,0.0,Distrito Federal,Miguel Hidalgo
1,51775,NaN,1.0,1.0,1.0,67.0,67.0,113851.0,21.032480,-89.592424,0.0,0.0,0.0,0.0,0.0,Yucatán,Mérida
2,115253,0.0,2.0,1.0,2.0,87.0,100.0,23620.0,19.332829,-99.152913,0.0,0.0,0.0,0.0,1.0,Distrito Federal,Coyoacán
3,299321,2.0,2.0,2.0,2.0,86.0,86.0,129347.0,16.860487,-99.878383,0.0,0.0,0.0,0.0,0.0,Guerrero,Acapulco de Juárez
4,173570,10.0,2.0,1.0,1.0,80.0,76.0,57125.0,19.640482,-99.127273,0.0,0.0,0.0,1.0,1.0,Edo. de México,Tultitlán


# Se codifica el tipo de propiedad

In [35]:
data['train'] = True
test['train'] = False
combined = pd.concat([data, test], sort = True)

In [36]:
len(combined)

300000

In [37]:
var_categoricas = ['tipodepropiedad']

In [38]:
one_hot_enc = ce.OneHotEncoder(handle_unknown = 'ignore')
one_hot_encoded = one_hot_enc.fit_transform(combined[var_categoricas])
one_hot_encoded.columns

Index(['tipodepropiedad_1', 'tipodepropiedad_2', 'tipodepropiedad_3',
       'tipodepropiedad_4', 'tipodepropiedad_5', 'tipodepropiedad_6',
       'tipodepropiedad_7', 'tipodepropiedad_8', 'tipodepropiedad_9',
       'tipodepropiedad_10', 'tipodepropiedad_11', 'tipodepropiedad_12',
       'tipodepropiedad_13', 'tipodepropiedad_14', 'tipodepropiedad_15',
       'tipodepropiedad_16', 'tipodepropiedad_17', 'tipodepropiedad_18',
       'tipodepropiedad_19', 'tipodepropiedad_20', 'tipodepropiedad_21',
       'tipodepropiedad_22', 'tipodepropiedad_23', 'tipodepropiedad_24',
       'tipodepropiedad_25'],
      dtype='object')

In [39]:
combined.columns

Index(['antiguedad', 'banos', 'centroscomercialescercanos', 'ciudad',
       'descripcion', 'direccion', 'escuelascercanas', 'fecha', 'garages',
       'gimnasio', 'habitaciones', 'id', 'idzona', 'lat', 'lng',
       'metroscubiertos', 'metrostotales', 'piscina', 'precio', 'provincia',
       'tipodepropiedad', 'titulo', 'train', 'usosmultiples'],
      dtype='object')

In [40]:
combined = pd.concat([combined, one_hot_encoded], axis = 1)
len(combined)

300000

In [41]:
combined.columns

Index(['antiguedad', 'banos', 'centroscomercialescercanos', 'ciudad',
       'descripcion', 'direccion', 'escuelascercanas', 'fecha', 'garages',
       'gimnasio', 'habitaciones', 'id', 'idzona', 'lat', 'lng',
       'metroscubiertos', 'metrostotales', 'piscina', 'precio', 'provincia',
       'tipodepropiedad', 'titulo', 'train', 'usosmultiples',
       'tipodepropiedad_1', 'tipodepropiedad_2', 'tipodepropiedad_3',
       'tipodepropiedad_4', 'tipodepropiedad_5', 'tipodepropiedad_6',
       'tipodepropiedad_7', 'tipodepropiedad_8', 'tipodepropiedad_9',
       'tipodepropiedad_10', 'tipodepropiedad_11', 'tipodepropiedad_12',
       'tipodepropiedad_13', 'tipodepropiedad_14', 'tipodepropiedad_15',
       'tipodepropiedad_16', 'tipodepropiedad_17', 'tipodepropiedad_18',
       'tipodepropiedad_19', 'tipodepropiedad_20', 'tipodepropiedad_21',
       'tipodepropiedad_22', 'tipodepropiedad_23', 'tipodepropiedad_24',
       'tipodepropiedad_25'],
      dtype='object')

In [42]:
train_set = train_set.merge(combined.loc[:, ['tipodepropiedad_1', 'tipodepropiedad_2', 'tipodepropiedad_3',
       'tipodepropiedad_4', 'tipodepropiedad_5', 'tipodepropiedad_6',
       'tipodepropiedad_7', 'tipodepropiedad_8', 'tipodepropiedad_9',
       'tipodepropiedad_10', 'tipodepropiedad_11', 'tipodepropiedad_12',
       'tipodepropiedad_13', 'tipodepropiedad_14', 'tipodepropiedad_15',
       'tipodepropiedad_16', 'tipodepropiedad_17', 'tipodepropiedad_18',
       'tipodepropiedad_19', 'tipodepropiedad_20', 'tipodepropiedad_21',
       'tipodepropiedad_22', 'tipodepropiedad_23', 'tipodepropiedad_24',
       'tipodepropiedad_25', 'id']], on='id', how = 'left')

In [43]:
test_set = test_set.merge(combined.loc[:, ['tipodepropiedad_1', 'tipodepropiedad_2', 'tipodepropiedad_3',
       'tipodepropiedad_4', 'tipodepropiedad_5', 'tipodepropiedad_6',
       'tipodepropiedad_7', 'tipodepropiedad_8', 'tipodepropiedad_9',
       'tipodepropiedad_10', 'tipodepropiedad_11', 'tipodepropiedad_12',
       'tipodepropiedad_13', 'tipodepropiedad_14', 'tipodepropiedad_15',
       'tipodepropiedad_16', 'tipodepropiedad_17', 'tipodepropiedad_18',
       'tipodepropiedad_19', 'tipodepropiedad_20', 'tipodepropiedad_21',
       'tipodepropiedad_22', 'tipodepropiedad_23', 'tipodepropiedad_24',
       'tipodepropiedad_25', 'id']], on='id', how = 'left')

In [44]:
len(train_set)

240000

In [45]:
len(test_set)

60000

# Se codifica la provincia

In [46]:
data['train'] = True
test['train'] = False
combined = pd.concat([data, test], sort = True)

In [47]:
len(combined)

300000

In [48]:
var_categoricas = ['provincia']

In [49]:
one_hot_enc = ce.OneHotEncoder(handle_unknown = 'ignore')
one_hot_encoded = one_hot_enc.fit_transform(combined[var_categoricas])
one_hot_encoded.columns

Index(['provincia_1', 'provincia_2', 'provincia_3', 'provincia_4',
       'provincia_5', 'provincia_6', 'provincia_7', 'provincia_8',
       'provincia_9', 'provincia_10', 'provincia_11', 'provincia_12',
       'provincia_13', 'provincia_14', 'provincia_15', 'provincia_16',
       'provincia_17', 'provincia_18', 'provincia_19', 'provincia_20',
       'provincia_21', 'provincia_22', 'provincia_23', 'provincia_24',
       'provincia_25', 'provincia_26', 'provincia_27', 'provincia_28',
       'provincia_29', 'provincia_30', 'provincia_31', 'provincia_32'],
      dtype='object')

In [50]:
combined.columns

Index(['antiguedad', 'banos', 'centroscomercialescercanos', 'ciudad',
       'descripcion', 'direccion', 'escuelascercanas', 'fecha', 'garages',
       'gimnasio', 'habitaciones', 'id', 'idzona', 'lat', 'lng',
       'metroscubiertos', 'metrostotales', 'piscina', 'precio', 'provincia',
       'tipodepropiedad', 'titulo', 'train', 'usosmultiples'],
      dtype='object')

In [51]:
combined = pd.concat([combined, one_hot_encoded], axis = 1)
len(combined)

300000

In [52]:
combined.columns

Index(['antiguedad', 'banos', 'centroscomercialescercanos', 'ciudad',
       'descripcion', 'direccion', 'escuelascercanas', 'fecha', 'garages',
       'gimnasio', 'habitaciones', 'id', 'idzona', 'lat', 'lng',
       'metroscubiertos', 'metrostotales', 'piscina', 'precio', 'provincia',
       'tipodepropiedad', 'titulo', 'train', 'usosmultiples', 'provincia_1',
       'provincia_2', 'provincia_3', 'provincia_4', 'provincia_5',
       'provincia_6', 'provincia_7', 'provincia_8', 'provincia_9',
       'provincia_10', 'provincia_11', 'provincia_12', 'provincia_13',
       'provincia_14', 'provincia_15', 'provincia_16', 'provincia_17',
       'provincia_18', 'provincia_19', 'provincia_20', 'provincia_21',
       'provincia_22', 'provincia_23', 'provincia_24', 'provincia_25',
       'provincia_26', 'provincia_27', 'provincia_28', 'provincia_29',
       'provincia_30', 'provincia_31', 'provincia_32'],
      dtype='object')

In [53]:
train_set = train_set.merge(combined.loc[:, ['provincia_1',
       'provincia_2', 'provincia_3', 'provincia_4', 'provincia_5',
       'provincia_6', 'provincia_7', 'provincia_8', 'provincia_9',
       'provincia_10', 'provincia_11', 'provincia_12', 'provincia_13',
       'provincia_14', 'provincia_15', 'provincia_16', 'provincia_17',
       'provincia_18', 'provincia_19', 'provincia_20', 'provincia_21',
       'provincia_22', 'provincia_23', 'provincia_24', 'provincia_25',
       'provincia_26', 'provincia_27', 'provincia_28', 'provincia_29',
       'provincia_30', 'provincia_31', 'provincia_32', 'id']], on='id', how = 'left')

In [54]:
test_set = test_set.merge(combined.loc[:, ['provincia_1',
       'provincia_2', 'provincia_3', 'provincia_4', 'provincia_5',
       'provincia_6', 'provincia_7', 'provincia_8', 'provincia_9',
       'provincia_10', 'provincia_11', 'provincia_12', 'provincia_13',
       'provincia_14', 'provincia_15', 'provincia_16', 'provincia_17',
       'provincia_18', 'provincia_19', 'provincia_20', 'provincia_21',
       'provincia_22', 'provincia_23', 'provincia_24', 'provincia_25',
       'provincia_26', 'provincia_27', 'provincia_28', 'provincia_29',
       'provincia_30', 'provincia_31', 'provincia_32', 'id']], on='id', how = 'left')

In [55]:
len(train_set)

240000

In [56]:
len(test_set)

60000

# Se codifica la ciudad

In [57]:
data['ciudad'].nunique()

875

### Se tienen 875 ciudades distintas. Por ende, usar One Hot es inviable

### Se prueba con Binary Encoding

In [58]:
data['train'] = True
test['train'] = False
combined = pd.concat([data, test], sort = True)

In [59]:
len(combined)

300000

In [60]:
var_categoricas = ['ciudad']

In [61]:
one_hot_enc = ce.BinaryEncoder(handle_unknown = 'ignore')
one_hot_encoded = one_hot_enc.fit_transform(combined[var_categoricas])
one_hot_encoded.columns

Index(['ciudad_0', 'ciudad_1', 'ciudad_2', 'ciudad_3', 'ciudad_4', 'ciudad_5',
       'ciudad_6', 'ciudad_7', 'ciudad_8', 'ciudad_9', 'ciudad_10'],
      dtype='object')

In [62]:
combined.columns

Index(['antiguedad', 'banos', 'centroscomercialescercanos', 'ciudad',
       'descripcion', 'direccion', 'escuelascercanas', 'fecha', 'garages',
       'gimnasio', 'habitaciones', 'id', 'idzona', 'lat', 'lng',
       'metroscubiertos', 'metrostotales', 'piscina', 'precio', 'provincia',
       'tipodepropiedad', 'titulo', 'train', 'usosmultiples'],
      dtype='object')

In [63]:
combined = pd.concat([combined, one_hot_encoded], axis = 1)
len(combined)

300000

In [64]:
combined.columns

Index(['antiguedad', 'banos', 'centroscomercialescercanos', 'ciudad',
       'descripcion', 'direccion', 'escuelascercanas', 'fecha', 'garages',
       'gimnasio', 'habitaciones', 'id', 'idzona', 'lat', 'lng',
       'metroscubiertos', 'metrostotales', 'piscina', 'precio', 'provincia',
       'tipodepropiedad', 'titulo', 'train', 'usosmultiples', 'ciudad_0',
       'ciudad_1', 'ciudad_2', 'ciudad_3', 'ciudad_4', 'ciudad_5', 'ciudad_6',
       'ciudad_7', 'ciudad_8', 'ciudad_9', 'ciudad_10'],
      dtype='object')

In [65]:
train_set = train_set.merge(combined.loc[:, ['ciudad_0',
       'ciudad_1', 'ciudad_2', 'ciudad_3', 'ciudad_4', 'ciudad_5', 'ciudad_6',
       'ciudad_7', 'ciudad_8', 'ciudad_9', 'ciudad_10', 'id']], on='id', how = 'left')

In [66]:
test_set = test_set.merge(combined.loc[:, ['ciudad_0',
       'ciudad_1', 'ciudad_2', 'ciudad_3', 'ciudad_4', 'ciudad_5', 'ciudad_6',
       'ciudad_7', 'ciudad_8', 'ciudad_9', 'ciudad_10', 'id']], on='id', how = 'left')

In [67]:
len(train_set)

240000

In [68]:
len(test_set)

60000

# Nuevos features

## OJO!!! Pensar muy bien si tiene sentido agregar ese feature o no... Porque agregar de más puede ser muy malo a priori. Lo más simple suele ser lo mejor!!!

## Fecha de publicación

In [69]:
data['fecha'] = pd.to_datetime(data['fecha'])
data['anio'] = data['fecha'].dt.year

In [70]:
data['anio'].value_counts()

2016    94038
2015    51470
2014    40572
2013    30386
2012    23534
Name: anio, dtype: int64

In [71]:
train_set['anio_publ'] = data['anio']

In [72]:
test_set['anio_publ'] = pd.to_datetime(test['fecha']).dt.year

### Feature: correlacion precio vs provincia

In [73]:
# dfgp = data.dropna(subset = ['garages']).groupby(['provincia'])['garages', 'precio'].corr().reset_index()
# dfgp.rename(columns={'precio':'corr_p'},inplace=True)
# dfgp = dfgp[ dfgp['level_1'] == 'garages']
# dfgp.sort_values(['corr_p'], ascending = False, inplace = True)
# dfgp

In [74]:
# train_set = train_set.merge(dfgp[['provincia','corr_p']], on = 'provincia', how = 'left')
# test_set = test_set.merge(dfgp[['provincia','corr_p']], on = 'provincia', how = 'left')

### Feature: correlacion garage vs precio por ciudad

In [75]:
# dfgpc = data
# dfgpc['n'] = 1
# dfgpc = dfgpc.groupby(['ciudad']).filter(lambda x: x['n'].sum() >= 500)
# dfgpc['ciudad'].value_counts()

In [76]:
# dfgpc = dfgpc.dropna(subset = ['garages']).groupby(['ciudad'])['garages', 'precio'].corr().reset_index()
# dfgpc = dfgpc[ dfgpc['level_1'] == 'garages']
# dfgpc.sort_values(['precio'], ascending = False, inplace = True)
# dfgpc.rename(columns={'precio':'corr_p_gar_ciu'},inplace=True)

In [77]:
# plt.figure(figsize=(10,17))
# g = sns.barplot(x = dfgpc['corr_p_gar_ciu'], y = dfgpc['ciudad'])
# g.set_title("Correlacion entre cant de garages y precio", fontsize=15)
# g.set_xlabel("Correlacion", fontsize=12)
# g.set_ylabel("ciudad", fontsize=12)

In [78]:
# train_set = train_set.merge(dfgpc[['ciudad','corr_p_gar_ciu']], on = 'ciudad', how = 'left')
# test_set = test_set.merge(dfgpc[['ciudad','corr_p_gar_ciu']], on = 'ciudad', how = 'left')

### Feature: según la descripción

In [80]:
dfdesc = data

In [81]:
def is_nan(x):
    return (x is np.nan or x != x)

In [82]:
def count_word_desc(desc):
    res = []
    if(is_nan(desc)):
        return [('',0)]  
    for word in str(desc).split(' '):
        if(len(word) > 4):
            res.append((word,1))
    return res

In [83]:
def desc_word_max(row):
    global i_count_word_max
    print("it... "+str(i_count_word_max))
    i_count_word_max = i_count_word_max + 1
    res = count_word_desc(row['descripcion'])
    if(len(res) == 0):
        return "0"
    df = pd.DataFrame(res)
    df.rename(columns={0:'word',1:'count'},inplace=True)
    df = df.groupby(by='word').sum().sort_values('count',ascending=False).reset_index()
    return (str(df['word'][0]),str(df['count'][0]))

## Genera un df con '(palabra,cantidad_apariciones)' para el campo 'descripcion'
## OJO! Demora mucho tiempo!

In [ ]:
# i_count_word_max = 0
#dfdesc_to_run = dfdesc.iloc[:,:]
# dfdesc_to_run = dfdesc
# dfdesc_to_run['word_count'] = dfdesc_to_run.apply(desc_word_max,axis=1)

In [ ]:
# dfdesc_to_run['max_word'] = dfdesc_to_run['word_count'].apply(lambda x: x[0])
# dfdesc_to_run['max_word_count'] = dfdesc_to_run['word_count'].apply(lambda x: x[1])
# dfdesc = dfdesc_to_run[['id','precio','max_word','max_word_count']]

In [ ]:
# df_desc_count = dfdesc.sort_values(by='precio',ascending=False)

In [ ]:
# df_desc_count.head()

In [ ]:
# df_desc_count.to_csv("df_desc_count.csv", index = False)

### Feature: existencia o no de palabra en descripcion

In [102]:
desc_parse = pd.read_csv("df_desc_count.csv", dtype = dtypes)
print("count: "+str(desc_parse.count()))
desc_parse.head()

count: id                240000
precio            240000
max_word          238381
max_word_count    240000
dtype: int64


,id,precio,max_word,max_word_count
0,153761,12525000.0,vista,2
1,181271,12521000.0,"acabados,",1
2,64975,12520000.0,recámara,2
3,279973,12516000.0,niveles,2
4,165404,12515000.0,adquirir,2


In [103]:
#Limpio el df un poco a mano
desc_parse = desc_parse[desc_parse['max_word'] != '&nbsp;']

In [104]:
desc_parse = desc_parse[desc_parse['max_word'] != 'ba&ntilde;o']

In [105]:
desc_parse = desc_parse[desc_parse['max_word'] != '\n</p><p>']

In [106]:
#df.replace({'A': r'^ba.$'}, {'A': 'new'}, regex=True)
desc_parse = desc_parse.replace({'max_word':'<p>casa'},{'max_word':'casa'})

In [107]:
desc_parse = desc_parse.replace({'max_word':'baño,'},{'max_word':'baño'})

In [108]:
desc_parse_g = desc_parse.groupby(by='max_word').sum().reset_index().sort_values('max_word_count',ascending=False)

In [110]:
print("count: "+str(desc_parse_g.count()))
desc_parse_g.head()

count: max_word          27521
id                27521
precio            27521
max_word_count    27521
dtype: int64


,max_word,id,precio,max_word_count
23484,cuenta,1096198019,1.939173e+10,18419
25996,planta,846083527,1.213207e+10,12734
23447,cuarto,733897069,2.136666e+10,11298
26955,tiene,453096902,9.050838e+09,8229
21516,baño,562339197,1.080776e+10,7128


In [111]:
#agrego columnas para cada palabra
n_cols = 10
desc_col = pd.DataFrame()
for row in desc_parse_g.head(n_cols)['max_word']:
    desc_col[row] = 0
desc_col['id'] = 0
desc_col

,cuenta,planta,cuarto,tiene,baño,departamento,excelente,cocina,casa,acabados,id


In [149]:
desc_parse2 = desc_parse.merge(desc_col,on='id',how='left')
desc_parse2.head()

,id,precio,max_word,max_word_count,cuenta,planta,cuarto,tiene,baño,departamento,excelente,cocina,casa,acabados
0,153761,12525000.0,vista,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,181271,12521000.0,"acabados,",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,64975,12520000.0,recámara,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,279973,12516000.0,niveles,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,165404,12515000.0,adquirir,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [150]:
desc_parse2 = desc_parse2.fillna(0)
desc_parse2.head()

,id,precio,max_word,max_word_count,cuenta,planta,cuarto,tiene,baño,departamento,excelente,cocina,casa,acabados
0,153761,12525000.0,vista,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,181271,12521000.0,"acabados,",1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,64975,12520000.0,recámara,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,279973,12516000.0,niveles,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,165404,12515000.0,adquirir,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [117]:
#desc_parse2.count()

In [118]:
#desc_parse_g.count()

In [155]:
def load_cols(row):
    global desc_parse3
    col_name = row['max_word']
    try:
        print(col_name)
        print(row[col_name])
        row[col_name] = 1
        print(row[col_name])
        desc_parse3.append(row) 
    except NameError:
        print('Col not found')
    except KeyError:
        print('Col error')

In [162]:
for index, row in desc_parse2.head(10).iterrows():
    col_name = row['max_word']
    try:
        print(col_name)
        print(row[col_name])
        row[col_name] = 1
        print(row[col_name])
    except NameError:
        print('Col not found')
    except KeyError:
        print('Col error')
desc_parse2.head(10)

vista
Col error
acabados,
Col error
recámara
Col error
niveles
Col error
adquirir
Col error
&aacute;reas
Col error
acabados
0.0
1
cuarto
0.0
1
excelente
0.0
1

cuarto
Col error


,id,precio,max_word,max_word_count,cuenta,planta,cuarto,tiene,baño,departamento,excelente,cocina,casa,acabados
0,153761,12525000.0,vista,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,181271,12521000.0,"acabados,",1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,64975,12520000.0,recámara,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,279973,12516000.0,niveles,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,165404,12515000.0,adquirir,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,193788,12500000.0,&aacute;reas,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,56501,12500000.0,acabados,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,94019,12500000.0,cuarto,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,246504,12500000.0,excelente,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,131489,12500000.0,\ncuarto,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [156]:
desc_parse3 = pd.DataFrame()
desc_parse2.head(10).apply(load_cols,axis=1)
desc_parse3.head(10)

vista
Col error
acabados,
Col error
recámara
Col error
niveles
Col error
adquirir
Col error
&aacute;reas
Col error
acabados
0.0
1
cuarto
0.0
1
excelente
0.0
1

cuarto
Col error


""


### Feature: total de antiguedad por anio

In [ ]:
# data.transpose()

In [ ]:
# ant = data
# ant['count'] = 1
# ant = ant.groupby(by='antiguedad').sum()[['count','habitaciones','id']]
#ant.sort_values(ascending=False).tail(5)
# ant = ant.reset_index()
# ant.head()

In [ ]:
# def hab_ant(row):
#     if(row['antiguedad'] <= 10 ):
#         return row['count'] * 1
#     return row['habitaciones'] * 0.5

In [ ]:
# ant2 = ant
# ant2['c_hab_ant'] = ant2.apply(hab_ant,axis=1)
# ant2.head(15)

In [ ]:
# data = data.drop(columns='count')

In [ ]:
# ant3 = ant2[['c_hab_ant','antiguedad']]
# ant3.head()

In [ ]:
# train_set = train_set.merge(ant3,on = 'antiguedad',how = 'left')
# train_set.transpose()

In [ ]:
# test_set = test_set.merge(ant3,on = 'antiguedad',how = 'left')
# test_set.transpose()

### Feature: antiguedad por provincia

In [ ]:
# data.transpose()

In [ ]:
#Mergea el la columna indicada en 'on', tomando el df pasado como parámetro(df_to_use)
#how_feature indica que tipo de join se hace.
#El merge lo hace sobre 'train_set' y 'test_set'
# def merge_df(df_to_use, on_feature,how_feature):
#     print("dataframe a usar:"+str(df_to_use))
#     print("feature a usar:"+on_feature)
   # print("[test_set] Cantidad init: "+str(len(test_set.columns)))
    #print("[train_set] Cantidad init: "+str(len(train_set.columns)))
#     test_set.merge(df_to_use, on = on_feature, how = how_feature)
#     train_set.merge(df_to_use, on = on_feature, how = how_feature)
    #print("[test_set] Cantidad init: "+str(len(test_set.columns)))
    #print("[train_set] Cantidad init: "+str(len(train_set.columns)))

In [ ]:
# df = data.groupby(by = 'provincia').agg({'antiguedad':'mean'}).reset_index()
# df.sort_values(['antiguedad'], ascending = False, inplace = True)
# df=df.rename(columns={"antiguedad":"antiguedad_prov"})
# df.head()

# data = data.merge(df,on='provincia',how='inner')
# data.transpose()

In [ ]:
# train_set.head()

In [ ]:
# train_set = train_set.merge(data[['id','antiguedad_prov']], on = 'id', how = 'left')
# test_set = test_set.merge(data[['id','antiguedad_prov']], on = 'id', how = 'left')

# train_set.transpose()

In [ ]:
# test_set.transpose()

### Feature: Usos multiples piscina gimnasio según tipo de casa

In [ ]:
# data.head(1)

In [ ]:
# usos = data[['id','gimnasio','piscina','usosmultiples']]

In [ ]:
# def factor_usos2(row):
#     print(row)

In [ ]:
# def factor_usos(row):
#     factor = 0
#     if(row['gimnasio'] == 1):
#         factor = factor + 0.1
#     if(row['piscina'] == 1):
#         factor = factor + 0.1
#     if(row['usosmultiples'] == 1):
#         factor = factor + 0.1
#     return factor

In [ ]:
# usos.head()

In [ ]:
#usos['factor_usos'] = usos.apply(factor_usos,axis=1)

In [ ]:
#train_set = train_set.merge(usos[['id','factor_usos']], on = 'id', how = 'left')
#test_set = test_set.merge(usos[['id','factor_usos']], on = 'id', how = 'left')

### Feature: Relación existe entre el precio, los metros cuadrados y la cantidad de habitaciones de los departamentos

In [ ]:
# Seteamos el precio en dolares
# prop = data
# cotizacion_hoy = 19.55
# prop['precio_USD'] = prop['precio'] / cotizacion_hoy

# print(prop['precio_USD'].head(2))
# print(prop['precio'].head(2))

# prop['precioXmt'] = prop['precio_USD']/prop['metrostotales']

In [ ]:
# deptos = prop[prop['tipodepropiedad'] == 'Apartamento']
# deptos[['antiguedad','precioXmt','metrostotales','precio_USD','habitaciones']].describe()
# Vemos que el 75% de los departamentos tiene hasta 3 habitaciones

In [ ]:
# (prop.groupby('habitaciones').count()['id'])/prop['habitaciones'].count()*100
# r = deptos[['id','precioXmt','metrostotales','habitaciones']]
# Sacamos los NaN
# r = r[~r['metrostotales'].isna()]
# Sacamos los que tienen menos de max_hab habitaciones, ya que son casos aislados
# max_hab = 5
# r = r[r['habitaciones'] < max_hab]
# r.head()

In [ ]:
# desc=r.sort_values(by=['precioXmt'],ascending=False).describe()

In [ ]:
# desc

In [ ]:
# mean_precioxmt = desc.loc['mean','precioXmt']

In [ ]:
# def factor_precioXmt(row):
#     print(row)
#     if(row > mean_precioxmt):
#         return 1
#     return 0.5

In [ ]:
# r.iloc[0,0] > mean_precioxmt

In [ ]:
# r['factor_precioXmt'] = r['precioXmt'].apply(factor_precioXmt)
#r.transpose().head()

In [ ]:
# r.head()

In [ ]:
# test_set.transpose()

In [ ]:
#train_set = train_set.merge(r[['id','factor_precioXmt']], on = 'id', how = 'left')
#test_set = test_set.merge(r[['id','factor_precioXmt']], on = 'id', how = 'left')

In [ ]:
#test_set.transpose()

In [ ]:
# Con los nan da mejores resultados!
#train_set = train_set.dropna(0)
#train_set.replace(0, np.nan, inplace=True)
#train_set.head()

In [ ]:
#test_set = test_set.dropna(0)
#test_set.replace(0, np.nan, inplace=True)
#test_set.head()

# Se exporta para entrenar

## Se sacan las columnas 'id' de train_set

In [ ]:
#test_set = test_set.loc[:,test_set.columns != 'id']
train_set = train_set.loc[:,train_set.columns != 'id']

## Remuevo la provincia del set de datos

In [ ]:
test_set = test_set.loc[:,test_set.columns != 'provincia']
train_set = train_set.loc[:,train_set.columns != 'provincia']

## Remuevo la ciudad del set de datos

In [ ]:
test_set = test_set.loc[:,test_set.columns != 'ciudad']
train_set = train_set.loc[:,train_set.columns != 'ciudad']

## NO hace falta eliminar NULLs

In [ ]:
len(train_set)

In [ ]:
train_set.to_csv("train_set_xgb.csv", index = False)

In [ ]:
test_set.head()

In [ ]:
len(test_set)

In [ ]:
test_set.to_csv("test_set_xgb.csv", index = False)